In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
import sqlite3

# Crear (o conectar) base de datos destino
conn = sqlite3.connect('ventas_etl_dia4.db')

print("Conexión a la base de datos creada correctamente")

Conexión a la base de datos creada correctamente


In [3]:
# Tabla clientes
conn.execute('''
    CREATE TABLE IF NOT EXISTS clientes (
        id_cliente INTEGER PRIMARY KEY,
        nombre TEXT NOT NULL,
        email TEXT UNIQUE,
        ciudad TEXT,
        fecha_registro DATE
    )
''')

# Tabla productos
conn.execute('''
    CREATE TABLE IF NOT EXISTS productos (
        id_producto INTEGER PRIMARY KEY,
        nombre TEXT NOT NULL,
        precio REAL NOT NULL,
        categoria TEXT
    )
''')

# Tabla ventas
conn.execute('''
    CREATE TABLE IF NOT EXISTS ventas (
        id_venta INTEGER PRIMARY KEY,
        id_cliente INTEGER,
        id_producto INTEGER,
        cantidad INTEGER NOT NULL,
        precio_unitario REAL NOT NULL,
        fecha_venta DATE,
        FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente),
        FOREIGN KEY (id_producto) REFERENCES productos(id_producto)
    )
''')

conn.commit()

print("Tablas creadas correctamente")

Tablas creadas correctamente


In [4]:
import pandas as pd
import numpy as np

# Datos de clientes
clientes_df = pd.DataFrame({
    'id_cliente': range(1, 6),
    'nombre': ['Ana García', 'Carlos López', 'María Rodríguez', 'Juan Pérez', 'Luis Martín'],
    'email': [
        'ana@email.com',
        'carlos@email.com',
        'maria@email.com',
        'juan@email.com',
        'luis@email.com'
    ],
    'ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Sevilla'],
    'fecha_registro': pd.date_range('2023-01-01', periods=5, freq='MS')
})

clientes_df

,id_cliente,nombre,email,ciudad,fecha_registro
0,1,Ana García,ana@email.com,Madrid,2023-01-01
1,2,Carlos López,carlos@email.com,Barcelona,2023-02-01
2,3,María Rodríguez,maria@email.com,Madrid,2023-03-01
3,4,Juan Pérez,juan@email.com,Valencia,2023-04-01
4,5,Luis Martín,luis@email.com,Sevilla,2023-05-01


In [5]:
# Datos de productos
productos_df = pd.DataFrame({
    'id_producto': range(101, 106),
    'nombre': ['Laptop', 'Mouse', 'Teclado', 'Monitor', 'Audífonos'],
    'precio': [1200, 25, 80, 300, 150],
    'categoria': ['Electrónica', 'Accesorios', 'Accesorios', 'Electrónica', 'Audio']
})

productos_df

,id_producto,nombre,precio,categoria
0,101,Laptop,1200,Electrónica
1,102,Mouse,25,Accesorios
2,103,Teclado,80,Accesorios
3,104,Monitor,300,Electrónica
4,105,Audífonos,150,Audio


In [6]:
# Datos de ventas (con errores intencionales)
np.random.seed(42)

ventas_df = pd.DataFrame({
    'id_venta': range(1, 21),
    'id_cliente': np.random.choice(range(1, 8), 20),     # IDs 6 y 7 NO existen
    'id_producto': np.random.choice(range(101, 108), 20), # IDs 106 y 107 NO existen
    'cantidad': np.random.randint(1, 5, 20),
    'precio_unitario': np.random.choice([1200, 25, 80, 300, 150], 20),
    'fecha_venta': pd.date_range('2024-01-01', periods=20, freq='D')
})

ventas_df

,id_venta,id_cliente,id_producto,cantidad,precio_unitario,fecha_venta
0,1,7,106,2,300,2024-01-01
1,2,4,106,2,25,2024-01-02
2,3,5,102,3,25,2024-01-03
3,4,7,104,2,1200,2024-01-04
4,5,3,105,3,25,2024-01-05
5,6,5,101,4,150,2024-01-06
6,7,5,104,3,25,2024-01-07
7,8,7,102,4,300,2024-01-08
8,9,2,106,4,300,2024-01-09
9,10,3,105,1,300,2024-01-10


In [7]:
# Función para cargar con validaciones
def cargar_con_validacion(df, tabla, conn, claves_foraneas=None):
    try:
        if claves_foraneas:
            for columna, tabla_ref, columna_ref in claves_foraneas:
                valores_validos = pd.read_sql(
                    f'SELECT {columna_ref} FROM {tabla_ref}', conn
                )[columna_ref].tolist()

                invalidos = ~df[columna].isin(valores_validos)

                if invalidos.any():
                    print(
                        f"⚠️ Advertencia: {invalidos.sum()} registros con {columna} inválido "
                        f"(no existe en {tabla_ref})"
                    )

                # Filtrar registros inválidos
                df = df[~invalidos]

        df.to_sql(tabla, conn, index=False, if_exists='append')
        print(f"✓ Cargados {len(df)} registros en {tabla}")
        return True

    except Exception as e:
        print(f"✗ Error cargando {tabla}: {e}")
        return False

In [8]:
cargar_con_validacion(clientes_df, 'clientes', conn)
cargar_con_validacion(productos_df, 'productos', conn)

✓ Cargados 5 registros en clientes
✓ Cargados 5 registros en productos


True

In [9]:
claves_ventas = [
    ('id_cliente', 'clientes', 'id_cliente'),
    ('id_producto', 'productos', 'id_producto')
]

cargar_con_validacion(ventas_df, 'ventas', conn, claves_ventas)

⚠️ Advertencia: 5 registros con id_cliente inválido (no existe en clientes)
⚠️ Advertencia: 4 registros con id_producto inválido (no existe en productos)
✓ Cargados 11 registros en ventas


True

In [10]:
# Verificar conteos finales
for tabla in ['clientes', 'productos', 'ventas']:
    count = pd.read_sql(
        f'SELECT COUNT(*) FROM {tabla}', conn
    ).iloc[0, 0]
    print(f"{tabla}: {count} registros")

clientes: 5 registros
productos: 5 registros
ventas: 11 registros


In [11]:
ventas_por_cliente = pd.read_sql('''
    SELECT c.nombre,
           COUNT(v.id_venta) AS num_ventas,
           SUM(v.cantidad * v.precio_unitario) AS total_ventas
    FROM clientes c
    LEFT JOIN ventas v ON c.id_cliente = v.id_cliente
    GROUP BY c.id_cliente, c.nombre
    ORDER BY total_ventas DESC
''', conn)

print("Ventas por cliente:")
ventas_por_cliente

Ventas por cliente:


,nombre,num_ventas,total_ventas
0,Luis Martín,5,4400.0
1,María Rodríguez,4,765.0
2,Juan Pérez,1,300.0
3,Carlos López,1,100.0
4,Ana García,0,NaN


In [12]:
import sqlite3
import pandas as pd

# Conectar a la base
conn = sqlite3.connect('ventas_etl.db')

In [13]:
clientes_sql = pd.read_sql('SELECT * FROM clientes', conn)
productos_sql = pd.read_sql('SELECT * FROM productos', conn)
ventas_sql = pd.read_sql('SELECT * FROM ventas', conn)

DatabaseError: Execution failed on sql 'SELECT * FROM clientes': no such table: clientes

In [14]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

,name
0,ventas_consolidadas


In [15]:
import sqlite3
import pandas as pd
import numpy as np

conn = sqlite3.connect('ventas_etl_dia4.db')

In [16]:
conn.execute('''
    CREATE TABLE clientes (
        id_cliente INTEGER PRIMARY KEY,
        nombre TEXT NOT NULL,
        email TEXT UNIQUE,
        ciudad TEXT,
        fecha_registro DATE
    )
''')

conn.execute('''
    CREATE TABLE productos (
        id_producto INTEGER PRIMARY KEY,
        nombre TEXT NOT NULL,
        precio REAL NOT NULL,
        categoria TEXT
    )
''')

conn.execute('''
    CREATE TABLE ventas (
        id_venta INTEGER PRIMARY KEY,
        id_cliente INTEGER,
        id_producto INTEGER,
        cantidad INTEGER NOT NULL,
        precio_unitario REAL NOT NULL,
        fecha_venta DATE,
        FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente),
        FOREIGN KEY (id_producto) REFERENCES productos(id_producto)
    )
''')

conn.commit()

OperationalError: table clientes already exists

In [17]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)

,name
0,clientes
1,productos
2,ventas


In [18]:
clientes_sql = pd.read_sql('SELECT * FROM clientes', conn)
productos_sql = pd.read_sql('SELECT * FROM productos', conn)
ventas_sql = pd.read_sql('SELECT * FROM ventas', conn)

In [19]:
ventas_por_cliente = pd.read_sql('''
    SELECT 
        c.id_cliente,
        c.nombre,
        COUNT(v.id_venta) AS num_ventas,
        SUM(v.cantidad * v.precio_unitario) AS total_ventas
    FROM clientes c
    LEFT JOIN ventas v 
        ON c.id_cliente = v.id_cliente
    GROUP BY c.id_cliente, c.nombre
    ORDER BY total_ventas DESC
''', conn)

In [20]:
resumen_carga = pd.DataFrame({
    'tabla': ['clientes', 'productos', 'ventas'],
    'registros_cargados': [
        len(clientes_sql),
        len(productos_sql),
        len(ventas_sql)
    ]
})

In [21]:
with pd.ExcelWriter(
    'evidencia_ETL_dia4_carga_sql.xlsx',
    engine='openpyxl'
) as writer:
    
    clientes_sql.to_excel(writer, sheet_name='clientes_final', index=False)
    productos_sql.to_excel(writer, sheet_name='productos_final', index=False)
    ventas_sql.to_excel(writer, sheet_name='ventas_final', index=False)
    
    ventas_por_cliente.to_excel(writer, sheet_name='ventas_por_cliente', index=False)
    resumen_carga.to_excel(writer, sheet_name='resumen_carga', index=False)